## Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import pandas as pd
import numpy as np
import sys
sys.path.append("./sources/") if "./sources/" not in sys.path else ''
import utils

progbar = utils.ProgressBar(elapsed_time=True)
progbar.update_progress(0)

[0:00:00][----------] 0.00% 

## Parameters

In [53]:
uf = "AC"
start_page = 1

## Setting Driver and Variables

In [3]:
driver = webdriver.Chrome()

In [58]:
url = "https://portal.cfm.org.br/index.php?option=com_medicos&nomeMedico=&ufMedico={}&crmMedico=&municipioMedico=&tipoInscricaoMedico=&situacaoMedico=&detalheSituacaoMedico=&especialidadeMedico=&areaAtuacaoMedico=&pagina=3"
url = url.format(uf)
driver.get(url)
result = re.search("Mostrando página \d de (\d+)", driver.page_source)
end_page = 15000 if result == None else int(result.groups()[0])

print ("Number of pages do crawl: ", end_page)

Number of pages do crawl:  242


## Running Crawler

In [62]:
if start_page <= 1:
    df = pd.DataFrame(columns=["page", "name", "crm", "state", "subscription_date", 
               "subscription_type", "status", "second_subscription", "address", "phone", 
               "photo_url"])
    
for page in range(start_page, end_page, 1):
    progbar.update_progress((page-1)/end_page)
    
    if (len(driver.find_elements_by_class_name("resultado-mobile-coluna")) == 0):
        input("No profiles found for page {}. Reenter the captcha:".format(page))
    
    perfis = driver.find_elements_by_class_name("resultado-mobile-coluna")
    photos = driver.find_elements_by_class_name("img-thumbnail")
    
    for i_perfil, perfil in enumerate(perfis):    
        dados = perfil.text.split('\n')     

        name = dados[0]

        result = re.search("\d+", dados[1])
        crm = None if not result else result.group()

        result = re.search("\w{2}$", dados[1])
        state = None if not result else result.group()

        result = re.search("Data de Inscrição: (.*)", dados[2])
        subscription_date = None if result == None else result.groups()[0]

        result = re.search("Inscrição: (.*)", dados[3])
        subscription_type = None if result == None else result.groups()[0]

        result = re.search("Situação: (.*)", dados[4])
        status = None if result == None else result.groups()[0]

        result = re.search("Inscrições em outro estado: (.*)", dados[5])
        second_subscription = None if result == None else result.groups()[0]
        
        result = re.search("Especialidades/Áreas de Atuação: (.*)", dados[6])
        specialty = None if result == None else result.groups()[0]

        result = re.search("Endereço: (.*)", dados[7])
        address = None if result == 0 else result.groups()[0]

        result = re.search("Telefone\(s\): (.*)", dados[8])
        phone = None if result == None else result.groups()[0]        

        photo_url = photos[i_perfil].get_attribute("src")
        
        df.loc[df.shape[0]] = [page, name, crm, state, subscription_date, 
               subscription_type, status, second_subscription, address, phone, photo_url]

    next_url = re.sub(r"&pagina=\d+", "&pagina={}".format(page+1), url)
    driver.get(next_url)

[0:07:16][##########] 99.17% 

In [66]:
print ("Number of extracted profiles for {}: {}".format(uf, df.shape[0]))

Number of extracted profiles for AC: 2410


In [67]:
df.to_csv("./data/profiles/df_{}.csv".format(uf), sep=';', index=None)

In [68]:
driver.close()

In [ ]:
# df["phone"].apply(lambda x: None if x == "Exibição não autorizada pelo médico." else x)
# df[df["phone"].astype(str) != 'None'].query("phone != 'Exibição não autorizada pelo médico.' & phone != None").shape